## Transfer Scene Viewing Eyetracking Files from USB to Backup Device (.ps1)

VERSION: `Scene_Eyetrack_Auto_Push`

### Advanced Explanation

This PowerShell script is designed for data synchronization between two directories, `$source` and `$destination`, with logging capabilities. It starts by ensuring a log file exists at `$logFile`, creating it if necessary and appending the creation time.

The script retrieves the last synchronization time from the log file, defaulting to `[datetime]::MinValue` if no previous sync is recorded. It then iterates through combinations of predefined `$blockTypes` and `$blockNumbers` to construct subdirectory paths under the source and destination.

For each block, it checks if the source path exists. If it does, it ensures the corresponding destination path also exists, creating it if not. The script then looks for new directories in the source path (created after the last sync time). If new directories are found, it prompts the user for each directory, asking whether to copy it to the destination. Copied directories are reported in the console.

After processing all blocks, the script updates the log file with the current sync time. If no new directories were found across all blocks, it outputs a message indicating so. The script concludes by waiting for the user to press Enter.

### User-Friendly Explanation

This script helps back up certain folders from one location on your computer to another. It checks folders named "Encoding_Block#" and "Retrieval_Block#" in a specific source directory and backs them up to a destination directory, only if they are new or have been updated since the last backup.

First, the script makes sure there's a log file where it records each backup's date and time. It then goes through each folder, checking if there's anything new to back up. If it finds new or updated folders, it asks if you want to back up each one, showing the folder's name and size. If you agree, it copies the folder to the backup location and notes this in the console.

After checking all the folders, the script updates the log file with the current date and time. If there were no new or updated folders, it tells you so. Finally, it waits for you to press Enter before it closes.

### Oversimplified Explantion

The script is designed so that files created after the last sync will be copied over from the USB to the Backupdata folder. (This means you could fix a folder name and the old version won't be copied over again). This script will push the session folders from each block to their corresponding block folders on the backup device. 

In [ ]:
$source = "E:\SceneViewing_DAY3" 
$destination = "C:\Backupdata\SceneViewing_DAY3" 
$logFile = "C:\Backupdata\SceneViewing_DAY3\DAY3_USB_Log.txt" 

if (-not (Test-Path -Path $logFile)) {
    New-Item -Path $logFile -ItemType File
    "Log Created: $(Get-Date)" | Out-File -FilePath $logFile
}

$lastSyncTime = (Get-Content -Path $logFile -Tail 1) -replace "Last Sync:", ""
if ([string]::IsNullOrWhiteSpace($lastSyncTime)) {
    $lastSyncTime = [datetime]::MinValue
} else {
    $lastSyncTime = [datetime]::Parse($lastSyncTime)
}

$blockTypes = @("Encoding", "Retrieval")
$blockNumbers = 1..4
$noNewFolders = $true

foreach ($type in $blockTypes) {
    foreach ($number in $blockNumbers) {
        $blockName = "${type}_Block$number"
        $sourcePath = Join-Path -Path $source -ChildPath $blockName
        $destinationPath = Join-Path -Path $destination -ChildPath $blockName

        if (-not (Test-Path -Path $sourcePath)) {
            Write-Output "Source path $sourcePath does not exist."
            continue
        }

        if (-not (Test-Path -Path $destinationPath)) {
            New-Item -Path $destinationPath -ItemType Directory
        }

        $newData = Get-ChildItem -Path $sourcePath -Directory | Where-Object { $_.CreationTime -gt $lastSyncTime }

        if ($newData.Count -eq 0) {
            Write-Output "No new session folders in $blockName since last synced."
            continue
        }

        $noNewFolders = $false

        foreach ($dir in $newData) {
            $dirSizeBytes = (Get-ChildItem -Path $dir.FullName -Recurse -File | Measure-Object -Property Length -Sum).Sum
            $dirSizeMB = [math]::Round($dirSizeBytes / 1MB, 2)

            $userInput = Read-Host "Do you want to push $($dir.Name) [Size: $($dirSizeMB) MB] [Y/N]"

            if ($userInput -eq "Y") {
                $destPath = Join-Path -Path $destinationPath -ChildPath $dir.Name
                Copy-Item -Path $dir.FullName -Destination $destPath -Recurse
                Write-Output "Copied $($dir.Name) to $destinationPath"
            }
        }
    }
}

if ($noNewFolders) {
    Write-Output "No new session folders across all blocks since last synced."
}

"Last Sync: $(Get-Date)" | Out-File -FilePath $logFile -Append

Read-Host -Prompt "Press Enter to Close"